# SPARQL query

In [69]:
import pandas as pd
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON, CSV

### Input parameters

In [70]:
#latitude
#longitude
#radius

#### Example setting

In [97]:
# endpoint
endpoint = 'https://query.wikidata.org/sparql'

# query variables
latitude  = '49.487777777'
longitude = '8.466111111'
#obj_class = '' 
radius    = '15'
limit     = '30'

### Query

In [98]:
query = '''
        prefix bd:       <http://www.bigdata.com/rdf#>
        prefix geo:      <http://www.opengis.net/ont/geosparql#>
        prefix wd:       <http://www.wikidata.org/entity/>
        prefix wdt:      <http://www.wikidata.org/prop/direct/>
        prefix wikibase: <http://wikiba.se/ontology#>


        SELECT ?xLabel ?x ?somebodyLabel ?somebody ?classLabel ?otherLocation ?dist
        WHERE
        {{
          BIND('Point({longitude} {latitude})'^^geo:wktLiteral AS ?currentLocation).
          SERVICE wikibase:around {{
              ?x wdt:P625 ?otherLocation. 
              bd:serviceParam wikibase:center ?currentLocation. 
              bd:serviceParam wikibase:radius '{radius}'. 
          }}
          # x is named after
          ?x wdt:P138 ?somebody .

          # which Class is x (e.g. Street, church)
          ?x wdt:P31  ?class . 
          
          # x is a Street
          #?x wdt:P31 wd:Q79007 .

          # hat Staat Deutschland
          # ?x wdt:P17  ?Q183 .
          
          # Retrieve Labels
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "de". }} 
          
          # Compute Distance
          # BIND(geof:distance(?otherLocation, ?currentLocation) as ?dist) 
        }} 

        LIMIT {limit}
        #ORDER BY ?dist
        
        '''.format(longitude = longitude, latitude = latitude, radius = radius, limit = limit)
print(query)


        prefix bd:       <http://www.bigdata.com/rdf#>
        prefix geo:      <http://www.opengis.net/ont/geosparql#>
        prefix wd:       <http://www.wikidata.org/entity/>
        prefix wdt:      <http://www.wikidata.org/prop/direct/>
        prefix wikibase: <http://wikiba.se/ontology#>


        SELECT ?xLabel ?x ?somebodyLabel ?somebody ?classLabel ?otherLocation ?dist
        WHERE
        {
          BIND('Point(8.466111111 49.487777777)'^^geo:wktLiteral AS ?currentLocation).
          SERVICE wikibase:around {
              ?x wdt:P625 ?otherLocation. 
              bd:serviceParam wikibase:center ?currentLocation. 
              bd:serviceParam wikibase:radius '15'. 
          }
          # x is named after
          ?x wdt:P138 ?somebody .

          # which Class is x (e.g. Street, church)
          ?x wdt:P31  ?class . 
          
          # x is a Street
          #?x wdt:P31 wd:Q79007 .

          # hat Staat Deutschland
          # ?x wdt:P17  ?Q183 .
          


In [102]:
# Initializing SPARQL Wrapper and querying
sparql = SPARQLWrapper(endpoint)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

variables = ['?xLabel', '?x' '?somebodyLabel' '?somebody' '?classLabel' '?otherLocation' '?dist']

#display(type(result))
#for result in results["results"]["bindings"]:
#    for i in variables:
#        print(result['x']["value"])

#results
df = pd.DataFrame(results['results']['bindings'])
for col in df.columns:
    df[col] = df[col].apply(lambda x: x['value'])
df.head(5)


,x,otherLocation,somebody,xLabel,somebodyLabel,classLabel
0,http://www.wikidata.org/entity/Q1723819,Point(8.61112 49.4717),http://www.wikidata.org/entity/Q180936,St. Gallus,Gallus,Kirchengebäude
1,http://www.wikidata.org/entity/Q1867056,Point(8.60811 49.4708),http://www.wikidata.org/entity/Q1286608,Lobdengau-Museum,Lobdengau,Museum
2,http://www.wikidata.org/entity/Q2145184,Point(8.55738 49.55607),http://www.wikidata.org/entity/Q16055,Viernheimer Dreieck,Viernheim,Autobahndreieck
3,http://www.wikidata.org/entity/Q2316559,Point(8.48053 49.562),http://www.wikidata.org/entity/Q45581,St.-Michael-Kirche,Michael,Kirchengebäude
4,http://www.wikidata.org/entity/Q2317509,Point(8.64878 49.4241),http://www.wikidata.org/entity/Q43982,St. Bartholomäus,Bartholomäus,Kirchengebäude
